In [ ]:
%%time 

%run ../ModelSEEDpy/modelseedpy/biochem/modelseed_justification.py --changes_path "MSDB_corrections2.json"

from datetime import datetime
print(datetime.now())

# model = justifyDB(msdb_path=".", primals_path="MSDB_justification_primals.json")

Defining variables and objective	Done after 8.61953125 minutes
Defining constraints	Done after 0.47265625 minutes
Constructing the model	

sys.excepthook is missing
Traceback (most recent call last):
  File "C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
MemoryError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
  File "C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
  File "C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
  File "C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 1041, in launch_instance
  File "C:\Users\Andrew Freiburger\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\kernelapp.py", line 711, in 

In [4]:
from json import load

with open("MSDB_corrections.json") as jsonIn:
    primals = load(jsonIn)

In [5]:
print(len(primals))
print(primals["cpd00001_0"])

20357
{'H': {'original': 2, 'proposed': 1.0}, 'O': {'original': 1, 'proposed': 0.5}}


In [2]:
from optlang import Model
from json import load, dump
with open("correct_MSDB.json") as jsonIn:
    model = load(jsonIn)
    
optlang_model = Model.from_json(model)
proposed_changes = {}
for rxn in optlang_model.reactions:
    for met in rxn.metabolites:
        for ele in met.elements:
            varName = f"{rxn.id}_{met.id}~{ele}"
            original_stoich = met.elements[ele]
            proposed_stoich = optlang_model.primal_values[varName]
            if original_stoich == proposed_stoich:  continue
            proposed_changes[varName] = {"Original": original_stoich, "Proposed": proposed_stoich}
print(proposed_changes)
with open("proposed_changes.json", "w") as out:
    dump(proposed_changes, out, indent=3)
# print([index for index, val in enumerate(primals.values()) if val != 0])

KeyboardInterrupt: 

In [38]:
locals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'from_local', 'msdb', '_i2', 'met', 'rxn', '_i3', '_i4', 'compound_dir', 'met_dir', '_i5', '_i6', '_i7', '__nonzero__', 'OptlangHelper', 'Bounds', 'tupVariable', 'tupConstraint', 'tupObjective', 'FeasibilityError', 'ObjectAlreadyDefinedError', 'process_time', 're', 'combine_elements', 'parse_primals', '_check_names', 'justifyDB', 'ArgumentParser', 'Path', 'parser', 'args', '_i8', '_i9', '_i10', '_i11', 'load', 'jsonIn', 'primals', '_i12', '_i13', 'dump', '_i14', 'proposed_changes', '_i15', '_i16', 'Model', 'model', 'optlang_model', '_i17', '_i18', 'var', '_i19', '_i20', '_i21', '_i22', 'met_ele', '_i23', 'ele', '_i24', '_i25', '_i26', 'val', '_i27', '_i28', '_i29', '_i30', '_i31', 'changes', 'varName', 'rxnID', 'metID_ele', 'metID', '_i32', '_i33', '_i34', '_i35', 

In [5]:
with open("MSDB_justification_primals.json") as jsonIn:
    primals = load(jsonIn)
    
from modelseedpy.biochem import from_local

msdb = from_local(".")

In [8]:
# print(dir(primals))
changes = {}
for varName, val in primals.items():
    # print(var, val)
    if "charge" not in varName:
        rxnID, metID_ele = varName.split("_", 1)
        metID, ele = metID_ele.split("~")
        # print(rxnID, metID, ele)
        met = msdb.compounds.get_by_id("_".join(metID.split("_")[:-1]))
        met_ele = met.elements[ele]
        if val != met_ele:
            if metID not in changes:
                changes[metID] = {}
            changes[metID][ele] = {"original": met_ele, "proposed": val}
        continue
    # print(varName)
    rxnID, metID = varName.split("_", 1)
    metID = metID.split("~")[0]
    # print(rxnID, metID, ele)
    met = msdb.compounds.get_by_id("_".join(metID.split("_")[:-1]))
    if val != met.charge:
        if metID not in changes:
            changes[metID] = {}
        changes[metID]["charge"] = {"original": met.charge, "proposed": val}
    print(changes)
    break

# print(dir(optlang_model), "\n")
# for var in optlang_model.variables:
#     rxn, met_ele = var.name.split("_", 1)
#     met, ele = met_ele.split("~")
#     print(rxn, met, ele)
#     break

{'cpd00001_0': {'H': {'original': 2, 'proposed': 1.0}, 'O': {'original': 1, 'proposed': 0.5}, 'charge': {'original': 0, 'proposed': -3.0}}}


In [7]:
print(len(changes))

23019


In [45]:
def isnumber(string):
    try:  float(string) ; return True
    except: print(string, end="\r") ; return False

broken_charges = 0
for met in msdb.compounds:
    if not isnumber(met.charge):
        broken_charges += 1
        
print(broken_charges)

0


In [23]:
model

NameError: name 'model' is not defined

In [1]:
from modelseedpy.biochem import from_local
msdb = from_local(".")

In [3]:
undefined_formula_cpd = msdb.compounds.get_by_id("cpd16153")
print(undefined_formula_cpd.elements)

{}


In [6]:
for met in msdb.compounds:
    compound_dir = dir(met)
    print(met.charge)
    break
    
for rxn in msdb.reactions:
    for met in rxn.metabolites:
        met_dir = dir(met)
        break
    break

print(set(compound_dir) - set(met_dir))
print(set(compound_dir).symmetric_difference(set(met_dir)))

0
set()
{'base_id'}


In [5]:
troublesome_rxn = msdb.reactions.get_by_id("rxn05145")
troublesome_rxn.reaction

'cpd00001_0 + cpd00002_0 + cpd00009_1 --> cpd00008_0 + 2 cpd00009_0 + cpd00067_0'

In [1]:
primals = model.primal_values

NameError: name 'primals' is not defined